In [19]:
import pandas as pd
import openai
import random
import time
from tqdm import tqdm
import json

qna_pair = pd.read_csv('Data/500-Umrah-related-question-answer-pair.csv')

# Separate the data by source
source_1 = qna_pair[qna_pair['Sumber'] == "Buku Tuntunan Manasik Haji dan Umrah 2024 Kemenag"]
source_2 = qna_pair[qna_pair['Sumber'] == "Fiqh Praktis Haji dan Umrah Oleh Abu Yusuf Akhmad Ja'far\n"]
source_3 = qna_pair[qna_pair['Sumber'] == "Panduan Ibadah Haji Sesuai Sunnah Nabi Oleh Abu Ubaidah Yusuf bin Mukhtar As-Sidawi"]

# Function to sample questions
def sample_questions():
    conversation_samples = []
    for i in range(50):
        source_1_sample = source_1.sample(4)['No '].tolist()
        source_2_sample = source_2.sample(3)['No '].tolist()
        source_3_sample = source_3.sample(3)['No '].tolist()
        conversation_samples.append([i + 1, source_1_sample + source_2_sample + source_3_sample])
    return conversation_samples

# Generate the conversation samples
conversation_samples = sample_questions()

# Convert to DataFrame
conversation_df = pd.DataFrame(conversation_samples, columns=['Conversation Number', 'Question Numbers'])

# Function to create a conversation prompt
def create_conversation_prompt(question_numbers, qna_pair):
    conversation = ""
    for q_num in question_numbers:
        question = qna_pair[qna_pair['No '] == q_num]['Pertanyaan '].values[0]
        answer = qna_pair[qna_pair['No '] == q_num]['Jawaban'].values[0]
        conversation += f"Q: {question}\nA: {answer}\n\n"
    return conversation

# Function to send API request to OpenAI
def send_api_request(prompt, system_message, api_key, retries=3, backoff_factor=1):
    openai.api_key = api_key
    attempt = 0
    while attempt < retries:
        try:
            response = openai.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=4096,
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error during API request (attempt {attempt + 1}): {e}")
            time.sleep(backoff_factor * (2 ** attempt))
            attempt += 1
    return None

def main():
    # Define the system message to explain the expectations
    system_message = """
    You are an AI assistant with expertise in Umrah preparations, rituals, costs, and general travel and tourism in Saudi Arabia. 
    Your task is to generate a natural conversation flow based on 10 provided question-and-answer pairs.

    Please write a conversation that incorporates all these question-and-answer pairs in a seamless and engaging manner. 
    The conversation should feel like a daily chat, with the AI assistant responding in a professional yet friendly tone. 
    Ensure smooth transitions between each question to maintain a coherent flow. 
    The responses should strictly adhere to the provided input text.

    The output format should strictly in a dictionary like this (no additional string before and after the dictionary): 
    
    {"messages":[
         {'role': 'system','content': (a summary of what the assistant is capable about within this conversation of 10 question-answer pairs, example:This assistant is knowledgable about Wukuf, Umrah preparation, and other rituals details)'}
         {"role": "assistant", "content": "Assalamu'alaikum! Bagaimana kabar Bapak/Ibu? Apa ada yang bisa saya bantu terkait persiapan Umrah atau pertanyaan lainnya?"}, 
         {"role": "user", "content": " Wa'alaikumsalam. Alhamdulillah, saya baik. Saya ingin tahu, bagaimana sih cara melaksanakan wukuf di Arafah?"}, 
         {"role": "assistant", "content": "Alhamdulillah, senang mendengarnya. Wukuf di Arafah adalah puncak ibadah haji yang dilakukan dengan berdiam diri di Padang Arafah pada 9 Dzulhijjah, mulai dari tergelincir matahari sampai terbenam. Wukuf ini sangat penting karena tanpa melaksanakannya, haji tidak sah. Ada hal lain tentang wukuf di Arafah yang ingin Bapak/Ibu ketahui?"}, 
         {"role": "user", "content": " Terima kasih informasinya. Kalau misalnya saya sakit selama umrah, apa yang harus saya lakukan?"}
         ... (continue until it covers all 10 questions mentioned in the input)
         ]}
    """

    # Send API request with OPENAI API Key
    api_key = ''

    # Iterate over each row in the conversation_df to generate and save conversations
    for index, row in tqdm(conversation_df.iterrows(), total=conversation_df.shape[0]):
        question_numbers = row['Question Numbers']
        prompt = create_conversation_prompt(question_numbers, qna_pair)
        conversation = send_api_request(prompt, system_message, api_key=api_key)

        # Create a DataFrame for the current conversation
        conversation_data = pd.DataFrame([[row['Conversation Number'], conversation]], columns=['Conversation Number', 'Conversation'])

        # Append the current conversation DataFrame to the CSV file
        conversation_data.to_csv('50_generated_conversations_290724-2am.csv', mode='a', index=False, header=False)

if __name__ == '__main__':
    main()

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [13:17<00:00, 15.96s/it]


In [55]:
conv_data = pd.read_excel('50_conversation_excel_cleaned.xlsx') #manually cleaned
succ, err = [], []
for i in range(1, 50):
    try:
        eval(conv_data.iloc[i, 1])
        succ.append(i)
    except:
        err.append(i)

In [56]:
err #to prove no error now

[]

In [57]:
for i in conv_data['Conversations'].head():
    display(eval(i))
    print('---'*40)

{'messages': [{'role': 'system',
   'content': 'This assistant is knowledgeable about Wukuf, Umrah preparations, and other ritual details.'},
  {'role': 'assistant',
   'content': "Assalamu'alaikum! Bagaimana kabar Bapak/Ibu? Apa ada yang bisa saya bantu terkait persiapan Umrah atau pertanyaan lainnya?"},
  {'role': 'user',
   'content': "Wa'alaikumsalam. Alhamdulillah, saya baik. Apa syarat wajib haji dan umrah?"},
  {'role': 'assistant',
   'content': 'Syarat wajib haji dan umrah adalah: 1) Islam, 2) Baligh (dewasa), 3) Berakal sehat, 4) Merdeka (bukan hamba sahaya), dan 5) Istit}a’ah (mampu). Apakah bapak/ibu ingin tahu lebih lanjut tentang salah satu syarat ini?'},
  {'role': 'user',
   'content': 'Terima kasih informasinya. Bagaimana cara memperbanyak bacaan Al-Quran selama umrah?'},
  {'role': 'assistant',
   'content': 'Halo bapak/ibu, untuk memperbanyak bacaan Al-Quran selama umrah, bawa selalu mushaf kecil, gunakan aplikasi Al-Quran di ponsel, dan tetapkan waktu khusus untuk m

------------------------------------------------------------------------------------------------------------------------


{'messages': [{'role': 'system',
   'content': 'This assistant is knowledgeable about Wukuf, Umrah preparation, and other rituals details.'},
  {'role': 'assistant',
   'content': "Assalamu'alaikum! Bagaimana kabar Bapak/Ibu? Apa ada yang bisa saya bantu terkait persiapan Umrah atau pertanyaan lainnya?"},
  {'role': 'user',
   'content': "Wa'alaikumsalam. Alhamdulillah, saya baik. Saya ingin tahu, bagaimana sih cara melaksanakan wukuf di Arafah?"},
  {'role': 'assistant',
   'content': 'Alhamdulillah, senang mendengarnya. Wukuf di Arafah adalah puncak ibadah haji yang dilakukan dengan berdiam diri di Padang Arafah pada 9 Dzulhijjah, mulai dari tergelincir matahari sampai terbenam. Wukuf ini sangat penting karena tanpa melaksanakannya, haji tidak sah. Ada hal lain tentang wukuf di Arafah yang ingin Bapak/Ibu ketahui?'},
  {'role': 'user',
   'content': 'Terima kasih informasinya. Kalau misalnya saya sakit selama umrah, apa yang harus saya lakukan?'},
  {'role': 'assistant',
   'content'

------------------------------------------------------------------------------------------------------------------------


{'messages': [{'role': 'system',
   'content': 'This assistant is knowledgeable about the rituals of Umrah and Hajj, including details on Wukuf, badal haji, selection of accommodation, the performance of Tawaf, rest places, rukun umrah, rules after reciting Talbiah, understanding Tahallul, common mistakes in ziyarah, and beneficial prayers, including Ayat Kursi.'},
  {'role': 'assistant',
   'content': "Assalamu'alaikum! Bagaimana kabar Bapak/Ibu? Apa ada yang bisa saya bantu terkait persiapan Umrah atau pertanyaan lainnya?"},
  {'role': 'user',
   'content': "Wa'alaikumsalam. Alhamdulillah, saya baik. Saya ingin tahu dulu bagaimana pelaksanaan badal haji."},
  {'role': 'assistant',
   'content': 'Badal haji dilaksanakan oleh petugas yang ditunjuk pemerintah dan dibiayai oleh pemerintah. Jemaah atau keluarga tidak dikenakan biaya dan akan menerima sertifikat badal haji sebagai bukti. Apakah ada hal lain tentang pelaksanaan badal haji yang ingin Bapak/Ibu ketahui?'},
  {'role': 'user',


------------------------------------------------------------------------------------------------------------------------


{'messages': [{'role': 'system',
   'content': 'This assistant is knowledgeable about Umrah preparations, rituals, costs, and general travel and tourism in Saudi Arabia.'},
  {'role': 'assistant',
   'content': "Assalamu'alaikum! Bagaimana kabar Bapak/Ibu? Apa ada yang bisa saya bantu terkait persiapan Umrah atau pertanyaan lainnya?"},
  {'role': 'user',
   'content': "Wa'alaikumsalam. Alhamdulillah, saya baik. Saya ingin tahu, apa saja keutamaan shalat di Masjid Nabawi?"},
  {'role': 'assistant',
   'content': 'Keutamaan shalat di Masjid Nabawi sangat tinggi, yaitu shalat di Masjid Nabawi lebih mulia nilainya 1.000 kali lipat dibandingkan shalat di masjid lain, kecuali di Masjidil Haram yang lebih mulia 100.000 kali lipat. Apakah ada hal lain tentang keutamaan shalat yang ingin bapak/ibu ketahui?'},
  {'role': 'user',
   'content': 'Itu sangat luar biasa. Kalau sunnah-sunnah tawaf itu apa saja?'},
  {'role': 'assistant',
   'content': 'Sunnah-sunnah tawaf meliputi: 1) Berlari-lari kec

------------------------------------------------------------------------------------------------------------------------


{'messages': [{'role': 'system',
   'content': 'This assistant is knowledgeable about Wukuf, Umrah preparation, and other rituals details'},
  {'role': 'assistant',
   'content': "Assalamu'alaikum! Bagaimana kabar Bapak/Ibu? Apa ada yang bisa saya bantu terkait persiapan Umrah atau pertanyaan lainnya?"},
  {'role': 'user',
   'content': "Wa'alaikumsalam. Alhamdulillah, saya baik. Saya ingin tahu, bagaimana sih cara melaksanakan wukuf di Arafah?"},
  {'role': 'assistant',
   'content': 'Alhamdulillah, senang mendengarnya. Wukuf di Arafah adalah puncak ibadah haji yang dilakukan dengan berdiam diri di Padang Arafah pada 9 Dzulhijjah, mulai dari tergelincir matahari sampai terbenam. Wukuf ini sangat penting karena tanpa melaksanakannya, haji tidak sah. Ada hal lain tentang wukuf di Arafah yang ingin Bapak/Ibu ketahui?'},
  {'role': 'user',
   'content': 'Terima kasih informasinya. Kalau misalnya saya sakit selama umrah, apa yang harus saya lakukan?'},
  {'role': 'assistant',
   'content':

------------------------------------------------------------------------------------------------------------------------


In [58]:
conv_data.head()

,Number,Conversations
0,1,"{'messages':[\n {'role': 'system','content'..."
1,2,"{""messages"":[\n {'role': 'system','content'..."
2,3,"{""messages"":[\n {'role': 'system', 'content..."
3,4,"{\n ""messages"":[\n {'role': 'system'..."
4,5,"{""messages"":[\n {'role': 'system', 'co..."


In [65]:
conv_list = [eval(i) for con in conv_data['Conversations']]

In [69]:
import json

# Specify your output file name
output_file = "50_conversation_json_file.jsonl"

# Open the file in write mode
with open(output_file, 'w', encoding='utf-8') as file:
    for entry in conv_list:
        # Convert the dictionary to a JSON string and write it to the file
        json_string = json.dumps(entry, ensure_ascii=False)
        file.write(json_string + '\n')  # Add a newline to separate entries